In [1]:
! pip install transformers datasets accelerate -qqq

In [2]:
import torch

In [3]:
def gpu_utilization():
    '''
    gpu에 메모리 할당이 얼마나 되어있는지 확인하는 함수
    1024 ** 3 을 통해 GB로 만들어줌
    '''
    if torch.cuda.is_available():
        memory = torch.cuda.memory_allocated() / (1024 **3)
        print(f"GPU 사용량: {memory: .3f} GB")
    else:
        print("GPU")

gpu_utilization()

GPU 사용량:  0.000 GB


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
def load_model_and_tokenizer(model_id):
    '''
    model과 token을 가져오는 함수
    torch_dtype = 'auto'는 gpu에 맞는 타입으로 모델을 변경해줌. T4인 경우 f16으로 변경
    device_map : 모델을 gpu를 0번부터 순차적으로 load 하겠다
    '''
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
    if torch.cuda.is_available():
        model.to('cuda')

    return tokenizer, model

In [6]:
%%capture
model_id = 'EleutherAI/polyglot-ko-1.3b'
tokenizer, model = load_model_and_tokenizer(model_id)

In [7]:
gpu_utilization()

GPU 사용량:  2.599 GB


In [8]:
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [9]:
def estimate_memory_of_gradients(model):
    '''
    model gradients 메모리 추정 함수

    '''
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            total_memory += param.grad.nelement() * param.grad.element_size() # nelemnt: 저장된 값의 수, element_size: 데이터 크기
    return total_memory

In [10]:
def estimate_memory_of_optimizer(optimizer):
    '''
    optimizer의 메모리 추정 함수
    '''
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                total_memory += v.nelement() * v.element_size()
    return total_memory

In [11]:
from datasets import Dataset
import numpy as np

def make_dummy_dataset():
    seq_len, dataset_size = 256, 64
    dummy_data = {
        'input_ids' : np.random.randint(100, 30000, (dataset_size, seq_len)),
        'labels' : np.random.randint(100, 30000, (dataset_size, seq_len)),
    }

    dataset = Dataset.from_dict(dummy_data)
    dataset.set_format("pt")
    return dataset

In [12]:
dataset = make_dummy_dataset()
batch_size = 4
train_dataloader = DataLoader(dataset, batch_size)

In [15]:
def train_model(model, dataloader):

    optimizer = AdamW(model.parameters(), lr = 0.001)
    for batch in dataloader:
        batch = {k : v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        gradients_memory = estimate_memory_of_gradients(model)
        optimizer_memory = estimate_memory_of_optimizer(optimizer)
        gpu_utilization()
        optimizer.zero_grad()

        print(f"옵티마이저 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")
        print(f"그래디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")
        break


In [16]:
train_model(model, train_dataloader)

GPU 사용량:  10.586 GB
옵티마이저 메모리 사용량: 4.961 GB
그래디언트 메모리 사용량: 2.481 GB


In [17]:
import gc

def cleanup():
    if 'model' in globals(): # cpu에서 학습에 사용되었던 메모리가 있으면 직접 삭제
        del globals()['model']
    if 'dataset' in globals():
        del globals()['dataset']
    gc.collect()  # python interpreter에서 사용하지 않는 객체를 탐지해서 수집
    torch.cuda.empty_cache() # gpu 캐시 비우기

#globals()를 호출하면 현재 모듈의 전역 변수와 그 값들이 담긴 딕셔너리를 반환합니다